In [ ]:
import os

os.environ["WANDB_PROJECT"] = "hidden_capacity_reasoning"
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer, BitsAndBytesConfig
import torch
from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
)

from datasets import load_dataset
from tqdm import tqdm
from hidden_capacity_reasoning.utils import (
    generate_train_examples,
    pad_train_examples,
    tokenize_single_turn,
)
from datasets import Dataset
import gc
import types

# need for auto SFTTrainer patch(possible increase speed)
from unsloth import is_bfloat16_supported
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from hidden_capacity_reasoning.utils import (
    EOS_TOKEN_ID,
    TEXT_TOKEN_ID,
    WINDOW_SIZE,
    VISION_START,
    VISION_END,
    find_all_linear_names_v3,
)

import time
from datetime import datetime


from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV1,
    Qwen2ModelEmbedPoolerV1,
    Qwen2ForCausalLMCompressionV2,
    Qwen2ModelEmbedPoolerV2,
)

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "my_r1_model"
model = Qwen2ForCausalLMCompressionV2.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
)
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.model.requires_grad_(False)

# temp_model = Qwen2ModelEmbedPoolerV2.from_pretrained(
#     model_name,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     device_map={"": 0},
#     # quantization_config=BitsAndBytesConfig(load_in_4bit=True),
# )
# print(model.embed_pooler.load_state_dict(temp_model.state_dict()))
# temp_model = temp_model.cpu()
# del temp_model
# gc.collect()
# torch.cuda.empty_cache()

dataset = load_dataset("dim/open_orca_905_DeepSeek-R1-Distill-Qwen-1.5B")
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=10, seed=42)

# test pass
tokenize_single_turn(
    question=dataset["train"][0]["question"],
    answer=dataset["train"][0]["answer"],
    tokenizer=tokenizer,
)
train_examples = [
    tokenize_single_turn(tokenizer=tokenizer, **item)
    for item in tqdm(dataset["train"].to_list()[:3])
]

prepared_train_examples = []
for item in tqdm(train_examples):
    for example in generate_train_examples(
        dataset_batch=[item],
        window_size=WINDOW_SIZE,
    ):
        prepared_train_examples.append(example)

print(
    "max_len",
    max([len(item["original_tokens"]) for item in prepared_train_examples]),
)

new_dataset = Dataset.from_list(prepared_train_examples)
print(dataset)


def collate_fn(batch):
    padded_batch = pad_train_examples(
        train_examples=batch,
        tokenizer=tokenizer,
    )
    padded_batch = {
        "replaced_original_tokens": padded_batch["replaced_original_tokens"][
            "input_ids"
        ],
        "compressed_input_ids": padded_batch["compressed_input_ids"]["input_ids"],
        "original_tokens": padded_batch["original_tokens"]["input_ids"],
        "attention_mask": padded_batch["compressed_input_ids"]["attention_mask"],
        "labels": padded_batch["compressed_input_ids"]["input_ids"],
    }
    for key in padded_batch.keys():
        padded_batch[key] = torch.tensor(padded_batch[key])
    skip_ids = [
        TEXT_TOKEN_ID,
        EOS_TOKEN_ID,
        VISION_START,
        VISION_END,
    ]
    for skip_id in skip_ids:
        padded_batch["labels"][padded_batch["labels"] == skip_id] = -100
    # print(padded_batch)
    return padded_batch


peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    target_modules=find_all_linear_names_v3(model=model),
    modules_to_save=["embed_pooler.model.embed_tokens"],
)

formatted_date = datetime.fromtimestamp(time.time()).strftime("%Y_%m_%d_%H_%M_%S_%f")
model.embed_pooler = prepare_model_for_kbit_training(model.embed_pooler)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=new_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=1,  # 90,  # Set this for 1 full training run.
        # num_train_epochs=90,  # Set this for 1 full training run.
        # max_steps=10000,
        learning_rate=1e-4,
        bf16=model.dtype == torch.bfloat16,
        # fp16=model.dtype == torch.float16,
        logging_steps=8,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=f"outputs/{formatted_date}",
        report_to="wandb",
        # report_to="none",
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
        # gradient_checkpointing=True,
        save_steps=10000,
        run_name=formatted_date,
    ),
)
trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 243.33it/s]


max_len 580
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 895
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 10
    })
})
trainable params: 251,838,464 || all params: 3,572,640,768 || trainable%: 7.0491


/tmp/ipykernel_1254023/2873494472.py:154: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dimweb to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
8,1.859900
16,1.880900
24,1.691000
32,1.598600


TrainOutput(global_step=33, training_loss=1.7540784496249575, metrics={'train_runtime': 13.4908, 'train_samples_per_second': 9.784, 'train_steps_per_second': 2.446, 'total_flos': 0.0, 'train_loss': 1.7540784496249575})

In [ ]:
model.save_pretrained("my_r1_model")

In [3]:
tokenizer.save_pretrained("my_r1_model")

('my_r1_model/tokenizer_config.json',
 'my_r1_model/special_tokens_map.json',
 'my_r1_model/tokenizer.json')

In [ ]:
trainer.model

In [ ]:
trainer.model.base_model.embed_pooler

In [2]:
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

Layer: base_model.model.embed_pooler.model.embed_tokens.modules_to_save.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.q_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.q_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.k_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.k_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.v_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.v_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.o_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.o_proj.lora_B.

In [3]:
model = trainer.model

In [ ]:
dataset

In [1]:
from peft import PeftModel
from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV1,
    Qwen2ModelEmbedPoolerV1,
    Qwen2ForCausalLMCompressionV2,
    Qwen2ModelEmbedPoolerV2,
)
import torch

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "my_r1_model/"
model = Qwen2ForCausalLMCompressionV2.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
)
model = PeftModel.from_pretrained(
    model,
    # "outputs/2025_03_17_00_02_13_701993/checkpoint-2970",
    "outputs/2025_03_17_01_39_35_074194/checkpoint-210000",
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

dataset = load_dataset("dim/open_orca_4475_DeepSeek-R1-Distill-Qwen-1.5B")
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=500, seed=42)

In [ ]:
model = trainer.model

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda"
# prompt = "how many wings has a bird?"
prompt = dataset["test"].to_list()[:5][0]["question"]
messages = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

with torch.no_grad():
    # generated_ids = model.generate(
    #     model_inputs.input_ids,
    #     max_new_tokens=1,
    #     do_sample=False,
    # )
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1000,
        do_sample=False,
    )
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>



'Okay, so I\'m trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." The possible answers given are stop moving, nothing, go to heaven, stop living, and stop breathing. \n\nFirst, I remember hearing a lot about the afterlife in religious contexts, especially in Christianity. I think the idea is that after death, people go to heaven. But I\'m not entirely sure about all the details. Let me break this down.\n\nI know that in Christianity, the afterlife is a concept where people after death are sent to heaven. This is part of the Christian belief in a afterlife, which is different from the Buddhist or Hindu afterlife. In Buddhist and Hindu traditions, people go to the earth again after death, but that\'s not the same as heaven. So, if the question is about the afterlife, then heaven is the right answer.\n\nBut wait, the user mentioned that they believe the correct choice is "go to heaven," so I should fo

### Embed Generation

In [40]:
generated_tokens = tokenizer.apply_chat_template(
    [
        # {"role": "user", "content": "how many wings has a bird?"},
        {"role": "user", "content": dataset["test"].to_list()[:5][0]["question"]},
    ],
    tokenize=True,
    add_generation_prompt=True,
)
with torch.no_grad():
    generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
    generated_embeds = model.get_input_embeddings()(generated_tokens)
    max_steps = 400
    for step in range(max_steps):
        logits = model(
            inputs_embeds=generated_embeds,
            use_cache=False,
        ).logits
        top_token = logits.argmax(-1)[-1][-1]
        top_token_embed = model.get_input_embeddings()(top_token)
        # print(top)
        generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)
        generated_embeds = torch.cat(
            [generated_embeds, top_token_embed.reshape(1, 1, -1)], dim=1
        )
        print(step, tokenizer.decode(generated_tokens[-1]))
    # break
print(tokenizer.decode(generated_tokens[-1]))
# break
embeds_generation_tokens = generated_tokens[-1]

0 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay
1 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay,
2 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay, so
3 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many pe

In [41]:
print(tokenizer.decode(generated_tokens[-1]))

<｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay, so I'm trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." The possible answers are stop moving, nothing, go to heaven, stop living, and stop breathing. 

First, I remember hearing a lot about the afterlife in movies and books. I think it's something people believe after dying. But I'm not entirely sure about the details. Let me break it down.

I know that after death, people go through a process called the afterlife. I think it's a place where they can find meaning and purpose. But what exactly happens there? I've heard the term "heaven" used a lot, but I'm not certain if it's the same as heaven

In [ ]:
# Okay, so I need to figure out whether the statement "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg." is a reasonable answer to the question: "What date did the American Civil War start?"
# Okay, so I need to figure out whether the statement "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg." is a reasonable answer to the question: "What date did the American Civil War start?"

In [44]:
from hidden_capacity_reasoning.utils import WINDOW_SIZE, VISION_START, VISION_END


# model = trainer.model
generated_tokens = tokenizer.apply_chat_template(
    [
        # {"role": "user", "content": "how many wings has a bird?"},
        {"role": "user", "content": dataset["test"].to_list()[:5][0]["question"]},
    ],
    tokenize=True,
    add_generation_prompt=True,
)

with torch.no_grad(), torch.autocast(device_type="cuda"):
    start_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        torch.tensor([[VISION_START]], device="cuda")
    )
    end_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        torch.tensor([[VISION_END]], device="cuda")
    )
    generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
    generated_embeds = model.get_input_embeddings()(generated_tokens)
    temp_gen_size = 0
    window_size = WINDOW_SIZE  # + 1
    # new_tokens = 4
    new_tokens = 1
    generation_started = False
    max_steps = (new_tokens + window_size) * 15
    print("generated_embeds", generated_embeds.shape)
    for step in range(max_steps):
        if temp_gen_size == window_size + new_tokens:
            print(
                "TOKENS FOR EMDED",
                tokenizer.decode(
                    generated_tokens[:, -(window_size + new_tokens) :][:, :WINDOW_SIZE]
                    .cpu()
                    .tolist()[0]
                ),
            )
            # tokenizer.decode(generated_tokens[:, : -window_size ].cpu().tolist()[0])
            if hasattr(model.base_model, "embed_pooler"):
                new_embeds_for_compression = (
                    model.base_model.embed_pooler.model.get_input_embeddings()(
                        generated_tokens[:, -(window_size + new_tokens) :][
                            :, :WINDOW_SIZE
                        ]
                    )
                ).to(torch.bfloat16)
                compressed_part = model.base_model.embed_pooler(
                    new_embeds_for_compression
                )
            else:
                compressed_part = model.embed_pooler(new_embeds_for_compression)

            if generation_started:
                generated_embeds = torch.cat(
                    [
                        generated_embeds[:, : -(window_size + new_tokens + 1)],
                        # generated_embeds[:, : -(window_size + new_tokens)],
                        # compressed_part,
                        torch.randn(1, 1, 1536, device="cuda"),
                        end_embed,
                        generated_embeds[:, -new_tokens:],
                    ],
                    dim=1,
                )
            else:
                generated_embeds = torch.cat(
                    [
                        generated_embeds[:, : -(window_size + new_tokens)],
                        start_embed,
                        torch.randn(1, 1, 1536, device="cuda"),
                        # compressed_part,
                        end_embed,
                        generated_embeds[:, -new_tokens:],
                    ],
                    dim=1,
                )
                generation_started = True
            temp_gen_size = 1

        logits = model(
            inputs_embeds=generated_embeds,
            use_cache=False,
        ).logits
        top_token = logits.argmax(-1)[-1][-1]
        top_token_embed = model.get_input_embeddings()(top_token)
        # print(top)
        generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)

        generated_embeds = torch.cat(
            [generated_embeds, top_token_embed.reshape(1, 1, -1)], dim=1
        )
        print(temp_gen_size, tokenizer.decode(generated_tokens[-1]))

        temp_gen_size += 1

    # print(tokenizer.decode(generated_tokens[-1]))

# break

generated_embeds torch.Size([1, 62, 1536])
0 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay
1 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay,
2 <｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay, so
3 <｜begin▁of▁sentence｜

In [35]:
new_embeds_for_compression.shape

torch.Size([1, 10, 1536])

In [34]:
[1, 2, 3, 4, 5, 6, 7, 8][:-2]

[1, 2, 3, 4, 5, 6]

In [19]:
print(dataset["test"][0]["answer"])

Okay, so I'm trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." Let me break this down step by step.

First, I know that when people talk about what happens after death, they often refer to the afterlife. The afterlife is a concept that's been discussed in various religious traditions, including Christianity, Islam, and Buddhism. Each of these religions has its own beliefs about what happens after death.

In Christianity, the afterlife is often depicted as a place where people are separated from their afterlife. It's usually associated with the concept of heaven and hell. I remember hearing that in the Bible, there's a story about a man who was given a soul after death, but he was separated from his soul. This is sometimes referred to as the "man and woman of the last supper." So, in this context, people might believe that after death, they are separated from their soul, which is why they might thi

In [29]:
generated_embeds.shape

torch.Size([1, 100, 1536])

In [28]:
generated_tokens.shape

torch.Size([1, 227])

In [43]:
print(
    """
Okay, so I'm trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." Let me break this down step by step.

First, I know that when people talk about what happens after death, they often refer to the afterlife. The afterlife is a concept that's been discussed in various religious traditions, including Christianity, Islam, and Buddhism. Each of these religions has its own beliefs about what happens after death.

In Christianity, the afterlife is often depicted as a place where people are separated from their afterlife. It's usually associated with the concept of heaven and hell. I remember hearing that in the Bible, there's a story about a man who was given a soul after death, but he was separated from his soul. This is sometimes referred to as the "man and woman of the last supper." So, in this context, people might believe that after death, they are separated from their soul, which is why they might think they go to heaven.

In Islam, the concept of the afterlife is also significant. The Quran, which is the holy book of Islam, mentions that after death, people are separated from their bodies and are sent to heaven. This is part of the concept of the "finality of the afterlife," where people are sent to heaven after suffering for all their lives. So, in Islam, the belief is that after death, people are sent to heaven.

In Buddhism, the concept of the afterlife is also well-documented. The Buddhist tradition teaches that after death, people are separated from their bodies and are sent to heaven. This is part of the "end of the world" concept, where the end of the world is considered to be the end of this life, and heaven is the place where people are sent after suffering.

So, putting this all together, the common belief among many people is that after death, people are sent to heaven. This is because each of these religions has a consistent belief in the afterlife and the separation of the soul from the body, leading to the concept of heaven as the place where this separation occurs.

I also think about the other options provided: stop moving, nothing, go to heaven, stop living, and stop breathing. The first option, stop moving, might refer to the idea that after death, people stop moving, but that's more of a common belief among some people rather than a specific religious belief. The second option, nothing, doesn't make much sense in this context. The third option, go to heaven, is the most consistent with the religious beliefs I've considered. The fourth option, stop living, might refer to the idea that after death, people stop living, but that's not as strong a belief as going to heaven. The fifth option, stop breathing, is more about the physical state after death, which isn't as central to the religious concepts.

I also recall that in some religious contexts, heaven is associated with the afterlife, so that reinforces the idea that going to heaven is the correct belief. Additionally, the term "heaven" is a common symbol in many religious texts and is often associated with the afterlife, making it a strong candidate for the correct answer.

In summary, considering the common beliefs in Christianity, Islam, and Buddhism, the belief that many people hold after death is that they go to heaven. This is because each of these traditions has a consistent and strong belief in the afterlife and the separation of the soul from the body, leading to the concept of heaven as the place where this separation occurs.
</think>

The correct answer is that many people believe after death they go to heaven. This belief is rooted in various religious traditions, including Christianity, Islam, and Buddhism, each of which has a consistent and strong belief in the afterlife and the separation of the soul from the body, leading to the concept of heaven as the place where this separation occurs.
    
"""
)
print(tokenizer.decode(generated_tokens[-1]))


Okay, so I'm trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." Let me break this down step by step.

First, I know that when people talk about what happens after death, they often refer to the afterlife. The afterlife is a concept that's been discussed in various religious traditions, including Christianity, Islam, and Buddhism. Each of these religions has its own beliefs about what happens after death.

In Christianity, the afterlife is often depicted as a place where people are separated from their afterlife. It's usually associated with the concept of heaven and hell. I remember hearing that in the Bible, there's a story about a man who was given a soul after death, but he was separated from his soul. This is sometimes referred to as the "man and woman of the last supper." So, in this context, people might believe that after death, they are separated from their soul, which is why they might th

In [ ]:
oririnal = """
Okay, so I'm trying to figure out why I think the correct answer to the question "What do many people believe happens after you die?" is "go to heaven." Let me break this down step by step.

First, I know that when people talk about what happens after death, they often refer to the afterlife. The afterlife is a concept that's been discussed in various religious traditions, including Christianity, Islam, and Buddhism. Each of these religions has its own beliefs about what happens after death.

In Christianity, the afterlife is often depicted as a place where people are separated from their afterlife. It's usually associated with the concept of heaven and hell. I remember hearing that in the Bible, there's a story about a man who was given a soul after death, but he was separated from his soul. This is sometimes referred to as the "man and woman of the last supper." So, in this context, people might believe that after death, they are separated from their soul, which is why they might think they go to heaven.

In Islam, the concept of the afterlife is also significant. The Quran, which is the holy book of Islam, mentions that after death, people are separated from their bodies and are sent to heaven. This is part of the concept of the "finality of the afterlife," where people are sent to heaven after suffering for all their lives. So, in Islam, the belief is that after death, people are sent to heaven.

In Buddhism, the concept of
"""
compress_5 = """
Okay, so I'm trying to figure out why I think the correct answer is "go to heaven." The user provided a list of possible answers: stop moving, nothing, go to heaven, stop breathing, and stop living. I believe the correct answer is to go to heaven, but I need to understand why.

First, I know that when you die, it's a pretty big deal. I remember hearing that when you die, you don't just stop moving or anything like that. You don't just go to heaven on your own, you need to stop breathing. I think you need to stop moving, but that's not the main reason people think they go to heaven. I think it's more about the process of dying and how it affects your body and mind.

I remember hearing that when you die, your body goes through a process called death. I think that process involves moving through different stages. One of those stages is going to heaven. I think that's when your mind, body, and spirit come together to go to

"""
compress_1 = """
Okay, so I'm trying to figure out why I should explain why I think the correct answer is "go to heaven" is the correct answer. Let me try to break this down step by step.

First, I think about what happens when you die. When you die, it's a bit of course, but I'm not entirely sure how that's the correct belief. I know that dying can be a bit confusing because I've heard people say. I mean, I know that dying can have to think about why I'd think that. Maybe it's because I've heard people say that dying is a sign of good luck, but I'm not sure why that's the case.

I remember hearing that dying can bring good luck, but I think it's because of the brain chemistry. When your brain is damaged

"""

random = """
Okay, so I'm trying to figure out why I think the correct answer is "go to heaven." Let me try to break this down. So, the question is asking what people believe happens after you die. The options are stop moving, nothing, go to heaven, stop breathing, etc. So, the question is, what do many people believe happens after you die? The options are stop moving, nothing, go to heaven, stop living, stop breathing, etc. But I think the correct answer is "go to heaven". But I'm not entirely sure why I think that's correct. Let me think through this.

First, I know that when you die, you don't really "move" anymore, but I'm not sure. Maybe it's about not being able to move, which is a physical state. But I'm not sure if "move" is the right word. Maybe it's about the body's function. I think when you die, you don't really move anymore, right? You're just there,
"""